# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/2_air_quality_feature_pipeline.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
!pip install geopy folium streamlit-folium --q

# Hosted notebook environments may not have the local features package
import os

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("⚙️ Downloading modules...")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/features/air_quality.py')
    os.system('wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/functions.py')
    os.system('wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/target_cities.json')
    print('✅ Done!')
else:
    print("Local environment")

In [ ]:
import datetime
import time
import requests
import pandas as pd
import json

from features import air_quality
from functions import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

In [ ]:
today = datetime.date.today()

In [ ]:
today, str(today)

---

## <span style='color:#ff5f27'> 🌫 Filling gaps in Air Quality data (PM2.5)</span>

### First time we will determine the 'last update date' using our backfill data
#### Next time we will use `feature view` method from Hopsworks Feature Store

In [ ]:
df_eu = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_eu.csv")
df_us = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_us.csv")
df_seattle = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_seattle.csv")

df_air_quality = pd.concat([df_eu, df_us, df_seattle])

In [ ]:
df_weather = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_weather.csv")

In [ ]:
last_dates_aq = df_air_quality[["date", "city_name"]].groupby("city_name").max()
last_dates_aq.date = last_dates_aq.date.astype(str)

# here is a dictionary with city names as keys and last updated date as values
last_dates_aq = last_dates_aq.to_dict()["date"]

In [ ]:
last_dates_aq["Paris"], last_dates_aq["Columbus"]

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing PM2.5 data

In [ ]:
start_of_cell = time.time()

df_aq_raw = pd.DataFrame()

for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                           coordinates=coords,
                                           start_date=last_dates_aq[city_name],
                                           end_date=str(today))
        df_aq_raw = pd.concat([df_aq_raw, df_]).reset_index(drop=True)
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new PM2.5 data for ALL locations up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

In [ ]:
# calculate 60 days ago from today
date_threshold = today - datetime.timedelta(days=60)

df_air_quality.date = (df_air_quality.date).astype(str)
# filter rows based on date threshold
df_air_quality = df_air_quality[df_air_quality['date'] > str(date_threshold)]

df_air_quality

In [ ]:
# we need the previous data to calculate aggregation functions
df_aq_update = pd.concat([df_air_quality, df_aq_raw]).reset_index(drop=True)
df_aq_update = df_aq_update.drop_duplicates(subset=['city_name', 'date'])

In [ ]:
df_aq_update.shape

### <span style="color:#ff5f27;">🛠 Feature Engineering PM2.5</span>

In [ ]:
df_aq_update['date'] = pd.to_datetime(df_aq_update['date'])

In [ ]:
df_aq_update = air_quality.feature_engineer_aq(df_aq_update)
df_aq_update = df_aq_update.dropna()

In [ ]:
df_aq_update.isna().sum().sum()

In [ ]:
df_aq_update.shape

In [ ]:
df_aq_update.columns

---

## <span style='color:#ff5f27'> 🌦 Filling gaps in Weather data</span>

In [ ]:
last_dates_weather = df_weather[["date", "city_name"]].groupby("city_name").max()
last_dates_weather.date = last_dates_weather.date.astype(str)
last_dates_weather = last_dates_weather.to_dict()["date"]

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing Weather data

In [ ]:
start_of_cell = time.time()

df_weather_update = pd.DataFrame()

for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        df_ = get_weather_data_from_open_meteo(city_name=city_name,
                                               coordinates=coords,
                                               start_date=last_dates_weather[city_name],
                                               end_date=str(today),
                                               forecast=True)
        df_weather_update = pd.concat([df_weather_update, df_]).reset_index(drop=True)
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new weather data for ALL cities up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

In [ ]:
df_aq_update.date = pd.to_datetime(df_aq_update.date)
df_weather_update.date = pd.to_datetime(df_weather_update.date)

df_aq_update["unix_time"] = df_aq_update["date"].apply(convert_date_to_unix)
df_weather_update["unix_time"] = df_weather_update["date"].apply(convert_date_to_unix)

In [ ]:
df_aq_update.date = df_aq_update.date.astype(str)
df_weather_update.date = df_weather_update.date.astype(str)

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather',
    version = 1
)

In [ ]:
air_quality_fg.insert(df_aq_update)

In [ ]:
weather_fg.insert(df_weather_update)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/3_air_quality_training_pipeline.ipynb)